In [ ]:
import sfacts as sf
import xarray as xr

In [ ]:
sim = sf.World.load('sim.world.nc')
fit = sf.World.load('sim.filt.fit.refit.world.nc')
fit = sf.World(fit.data.assign_coords(position=fit.position.astype(int)))

sim = sim.sel(position=fit.position.astype(int), sample=fit.sample.astype(int))

In [ ]:
mgen_error = sf.evaluation.metagenotype_error2(fit, discretized=True)
fwd_genotype_error = sf.evaluation.discretized_weighted_genotype_error(sim, fit)
rev_genotype_error = sf.evaluation.discretized_weighted_genotype_error(fit, sim)
bc_error = sf.evaluation.braycurtis_error(sim, fit)
unifrac_error = sf.evaluation.unifrac_error(sim, fit)
entropy_error = sf.evaluation.community_entropy_error(sim, fit)

In [ ]:
sf.plot.plot_community(
    sim,
    col_linkage_func=lambda w: sim.communities.linkage('sample'),
    col_colors_func=lambda w: xr.Dataset(dict(
        mg=mgen_error[1],
        bc=bc_error[1],
        uf=unifrac_error[1],
        ent=entropy_error[1],
    )),
    row_colors_func=lambda w: xr.Dataset(dict(
        fwd=fwd_genotype_error[1],
    )),
)
sf.plot.plot_community(
    fit,
    col_linkage_func=lambda w: sim.communities.linkage('sample'),
    col_colors_func=lambda w: xr.Dataset(dict(
        mg=mgen_error[1],
        bc=bc_error[1],
        uf=unifrac_error[1],
        ent=entropy_error[1],
    )),
    row_colors_func=lambda w: xr.Dataset(dict(
        rev=rev_genotype_error[1],
    )),
)
sf.plot.plot_metagenotype(
    fit.isel(position=slice(0, 500)),
    col_linkage_func=lambda w: sim.communities.linkage('sample'),
    col_colors_func=lambda w: xr.Dataset(dict(
        mg=mgen_error[1],
        bc=bc_error[1],
        uf=unifrac_error[1],
        ent=entropy_error[1],
    )),
    row_colors_func=lambda w: xr.Dataset(dict(
        _=w.metagenotypes.mean_depth("position"),
    )),
)
sf.plot.plot_genotype(
    fit.isel(position=slice(0, 500)),
    row_colors_func=lambda w: xr.Dataset(dict(
        rev=rev_genotype_error[1],
    )),
)

In [ ]:
%%bash
sfacts evaluate_fit sim.world.nc sim.filt.fit.refit.world.nc sim.filt.fit.refit.eval.tsv
cat sim.filt.fit.refit.eval.tsv